In [20]:
import requests
import lxml.html
import re
import tqdm
from selenium import webdriver
from selenium.webdriver import ActionChains
import pandas as pd
import time

In [21]:
url_csv = pd.read_csv("식당URL.csv")

In [22]:
driver = webdriver.Chrome('/Users/hanuel/Desktop/chromedriver')
review_list = []

In [23]:
for url, city in tqdm.tqdm(zip(url_csv['url'][:1694], url_csv['도시'][:10])):
    try:
        driver.get(url)
        driver.implicitly_wait(10)

        # 도시
        address = city

        # 식당이름
        tmp = driver.find_element_by_css_selector("div._1hkogt_o > h1")
        title = tmp.text

        try:
            # more element 를 찾음
            more = driver.find_element_by_xpath('//*[@id="taplc_location_reviews_list_resp_rr_resp_0"]/div/div[13]/div/div/a[2]')

            # more 까지 스크롤
            action = ActionChains(driver)
            action.move_to_element(more).perform()
            driver.implicitly_wait(10)

            total_pages = len(driver.find_elements_by_css_selector('div.pageNumbers > a'))

            driver.implicitly_wait(10)
        except:
            total_pages = 0

        try:
            tmp = driver.find_elements_by_css_selector("span.taLnk.ulBlueLinks")
            if len(tmp) >= 1:
                action = ActionChains(driver)
                action.move_to_element(tmp[0]).perform()
                driver.implicitly_wait(10)
                time.sleep(1)
                tmp = driver.find_elements_by_css_selector("span.taLnk.ulBlueLinks")
                tmp[0].click()
                driver.implicitly_wait(10)
                time.sleep(1)
        except:
            print("there is no 더보기")

        reviews = driver.find_element_by_css_selector("#taplc_location_reviews_list_resp_rr_resp_0")
        for review in reviews.find_elements_by_css_selector(".review-container"):
            try:
                review_date = review.find_element_by_css_selector(".ratingDate").text.split("에")[0]
                reviewer_id = review.find_element_by_css_selector("div.member_info").text.split("\n")[0]
                review_rate = int(review.find_element_by_css_selector("div.ui_column.is-9 > span").get_attribute("class")[-2])

                review_text = review.find_element_by_css_selector("div.entry > p.partial_entry").text

                dictionary = {}
                dictionary['region'] = address
                dictionary['title'] = title
                dictionary['review_date'] = review_date
                dictionary['review_id'] = reviewer_id
                dictionary['review_rate'] = review_rate
                dictionary['review_text'] = review_text
                review_list.append(dictionary)
            except:
                continue


        for i in range(total_pages - 1):
            next_ = driver.find_elements_by_css_selector('div.unified.ui_pagination a')[1]
            action = ActionChains(driver)
            action.move_to_element(next_).perform()
            next_.click()

            try:
                tmp = driver.find_elements_by_css_selector("span.taLnk.ulBlueLinks")
                if len(tmp) >= 1:
                    action = ActionChains(driver)
                    action.move_to_element(tmp[0]).perform()
                    driver.implicitly_wait(10)
                    time.sleep(1)
                    tmp = driver.find_elements_by_css_selector("span.taLnk.ulBlueLinks")
                    tmp[0].click()
                    driver.implicitly_wait(10)
                    time.sleep(1)
            except:
                print("there is no 더보기")

            reviews = driver.find_element_by_css_selector("#taplc_location_reviews_list_resp_rr_resp_0")
            for review in reviews.find_elements_by_css_selector(".review-container"):
                try:
                    review_date = review.find_element_by_css_selector(".ratingDate").text.split("에")[0]
                    reviewer_id = review.find_element_by_css_selector("div.member_info").text.split("\n")[0]
                    review_rate = int(review.find_element_by_css_selector("div.ui_column.is-9 > span").get_attribute("class")[-2])

                    review_text = review.find_element_by_css_selector("div.entry > p.partial_entry").text

                    dictionary = {}
                    dictionary['region'] = address
                    dictionary['title'] = title
                    dictionary['review_date'] = review_date
                    dictionary['review_id'] = reviewer_id
                    dictionary['review_rate'] = review_rate
                    dictionary['review_text'] = review_text
                    review_list.append(dictionary)
                except:
                    continue
        print(len(review_list))
    except:
        continue

1it [01:38, 98.26s/it]

70


2it [03:25, 103.30s/it]

160


3it [05:11, 104.69s/it]

...

8it [12:21, 84.01s/it]

591


9it [13:47, 84.59s/it]

661


10it [15:10, 91.02s/it]


In [24]:
ta_reviews_2 = pd.DataFrame(review_list)
ta_reviews_2.to_csv("ta_reviews2.csv", index = False)